In [ ]:
import pandas as pd
import torch
data_train = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/train.csv.gz', compression='gzip',header = None)
data_valid = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/valid.csv.gz', compression='gzip',header = None)
data_test = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/test.csv.gz', compression='gzip',header = None)
data, _ = torch.load(r"/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/processed/geometric_data_processed.pt")

# Extract edges for "paper" -> "cites" -> "paper"
paper_c_paper = data.edge_index_dict[('paper', 'cites', 'paper')]

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(paper_c_paper[0], nums_train) | torch.isin(paper_c_paper[1], nums_train)
mask_valid = torch.isin(paper_c_paper[0], nums_valid) | torch.isin(paper_c_paper[1], nums_valid)
mask_test = torch.isin(paper_c_paper[0], nums_test) | torch.isin(paper_c_paper[1], nums_test)

paper_c_paper_train = paper_c_paper.clone()
paper_c_paper_valid = paper_c_paper.clone()
paper_c_paper_test = paper_c_paper.clone()

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
paper_c_paper_train = paper_c_paper_train[:, mask_train_done]
paper_c_paper_valid = paper_c_paper_valid[:, mask_valid_done]
paper_c_paper_test = paper_c_paper_test[:, mask_test]

len(paper_c_paper_train[1]) + len(paper_c_paper_valid[1]) + len(paper_c_paper_test[1]), paper_c_paper.shape[1]


/tmp/ipykernel_37979/280286001.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data, _ = torch.load(r"/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/datase

(5416271, 5416271)

In [23]:
author_w_paper = data.edge_index_dict[('author', 'writes', 'paper')]

author_w_paper_train = author_w_paper.clone()
author_w_paper_valid = author_w_paper.clone()
author_w_paper_test = author_w_paper.clone()

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(author_w_paper[1], nums_train)
mask_valid = torch.isin(author_w_paper[1], nums_valid)
mask_test = torch.isin(author_w_paper[1], nums_test)

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
author_w_paper_train = author_w_paper_train[:, mask_train_done]
author_w_paper_valid = author_w_paper_valid[:, mask_valid_done]
author_w_paper_test = author_w_paper_test[:, mask_test]

len(author_w_paper_train[1]) + len(author_w_paper_valid[1]) + len(author_w_paper_test[1]), author_w_paper.shape[1]


(7145660, 7145660)

In [31]:
author_a_institution = data.edge_index_dict[('author', 'affiliated_with', 'institution')]

author_a_institution_train = author_a_institution.clone()
author_a_institution_valid = author_a_institution.clone()
author_a_institution_test = author_a_institution.clone()

nums_test = torch.tensor(author_w_paper_test[0].unique())
nums_valid = torch.tensor(author_w_paper_valid[0].unique())
nums_train = torch.tensor(author_w_paper_train[0].unique())

mask_train = torch.isin(author_a_institution[0], nums_train)
mask_valid = torch.isin(author_a_institution[0], nums_valid)
mask_test = torch.isin(author_a_institution[0], nums_test)

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
author_a_institution_train = author_a_institution_train[:, mask_train_done]
author_a_institution_valid = author_a_institution_valid[:, mask_valid_done]
author_a_institution_test = author_a_institution_test[:, mask_test]

len(author_a_institution_train[1]) + len(author_a_institution_valid[1]) + len(author_a_institution_test[1]), author_a_institution.shape[1]

/tmp/ipykernel_37979/2843819996.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  nums_test = torch.tensor(author_w_paper_test[0].unique())
/tmp/ipykernel_37979/2843819996.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  nums_valid = torch.tensor(author_w_paper_valid[0].unique())
/tmp/ipykernel_37979/2843819996.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  nums_train = torch.tensor(author_w_paper_train[0].unique())


(1043998, 1043998)

In [25]:
paper_t_field = data.edge_index_dict[('paper', 'has_topic', 'field_of_study')]

paper_t_field_train = paper_t_field.clone()
paper_t_field_valid = paper_t_field.clone()
paper_t_field_test = paper_t_field.clone()

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(paper_t_field[0], nums_train)
mask_valid = torch.isin(paper_t_field[0], nums_valid)
mask_test = torch.isin(paper_t_field[0], nums_test)

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
paper_t_field_train = paper_t_field_train[:, mask_train_done]
paper_t_field_valid = paper_t_field_valid[:, mask_valid_done]
paper_t_field_test = paper_t_field_test[:, mask_test]

len(paper_t_field_train[1]) + len(paper_t_field_valid[1]) + len(paper_t_field_test[1]), paper_t_field.shape[1]


(7505078, 7505078)

In [32]:
# Function to process edges based on train/valid/test masks
def process_edges(data, edge_type, nums_train, nums_valid, nums_test, check_both=True):
    edge_data = data.edge_index_dict[edge_type]

    # If check_both is True, check both parts of the edge
    if check_both:
        mask_train = torch.isin(edge_data[0], nums_train) | torch.isin(edge_data[1], nums_train)
        mask_valid = torch.isin(edge_data[0], nums_valid) | torch.isin(edge_data[1], nums_valid)
        mask_test = torch.isin(edge_data[0], nums_test) | torch.isin(edge_data[1], nums_test)
    else:
        # If check_both is False, check only one part (e.g., edge_data[0] or edge_data[1])
        mask_train = torch.isin(edge_data[0], nums_train)
        mask_valid = torch.isin(edge_data[0], nums_valid)
        mask_test = torch.isin(edge_data[0], nums_test)
    
    # Combine the conditions into a single mask that selects only the train edges
    mask_train_done = mask_train & ~mask_valid & ~mask_test
    mask_valid_done = mask_valid & ~mask_test

    edge_train = edge_data.clone()[:, mask_train_done]
    edge_valid = edge_data.clone()[:, mask_valid_done]
    edge_test = edge_data.clone()[:, mask_test]

    return edge_train, edge_valid, edge_test, edge_data

# Example usage where you only care about checking edge_data[0] (source node):
paper_c_paper_train, paper_c_paper_valid, paper_c_paper_test, paper_c_paper = process_edges(
    data, ('paper', 'cites', 'paper'), nums_train, nums_valid, nums_test)


In [3]:
len(paper_c_paper_train[1]) + len(paper_c_paper_valid[1]) + len(paper_c_paper_test[1]), paper_c_paper.shape[1]

(5416271, 5416271)

In [4]:
len(paper_c_paper_train[1]) + len(paper_c_paper_valid[1]) + len(paper_c_paper_test[1]), paper_c_paper.shape[1]

(5416271, 5416271)

In [7]:
print(len(paper_c_paper_train[1]) + len(paper_c_paper_valid[1]) + len(paper_c_paper_test[1]), paper_c_paper.shape[1])
print(len(author_w_paper_train[1]) + len(author_w_paper_valid[1]) + len(author_w_paper_test[1]), author_w_paper.shape[1])
print(len(author_a_institution_train[1]) + len(author_a_institution_valid[1]) + len(author_a_institution_test[1]), author_a_institution.shape[1])
print(len(paper_t_field_train[1]) + len(paper_t_field_valid[1]) + len(paper_t_field_test[1]), paper_t_field.shape[1])


5416271 5416271
7145660 7145660
1043998 1043998
7505078 7505078


In [6]:
import pandas as pd
import torch
data_train = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/train.csv.gz', compression='gzip',header = None)
data_valid = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/valid.csv.gz', compression='gzip',header = None)
data_test = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/test.csv.gz', compression='gzip',header = None)
data, _ = torch.load(r"/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/processed/geometric_data_processed.pt")

def process_edges(data, edge_type, nums_train, nums_valid, nums_test, mask_position):
    edge_data = data.edge_index_dict[edge_type]

    # If check_both is True, check both parts of the edge
    if mask_position == 'both':
        mask_train = torch.isin(edge_data[0], nums_train) | torch.isin(edge_data[1], nums_train)
        mask_valid = torch.isin(edge_data[0], nums_valid) | torch.isin(edge_data[1], nums_valid)
        mask_test = torch.isin(edge_data[0], nums_test) | torch.isin(edge_data[1], nums_test)
    elif mask_position == 'source':
        mask_train = torch.isin(edge_data[0], nums_train)
        mask_valid = torch.isin(edge_data[0], nums_valid)
        mask_test = torch.isin(edge_data[0], nums_test)
    elif mask_position == 'target':
        mask_train = torch.isin(edge_data[1], nums_train)
        mask_valid = torch.isin(edge_data[1], nums_valid)
        mask_test = torch.isin(edge_data[1], nums_test)
    else:
        raise ValueError('mask_position must be one of "both", "source", or "target"')
    
    # Combine the conditions into a single mask that selects only the train edges
    mask_train_done = mask_train & ~mask_valid & ~mask_test
    mask_valid_done = mask_valid & ~mask_test

    edge_train = edge_data.clone()[:, mask_train_done]
    edge_valid = edge_data.clone()[:, mask_valid_done]
    edge_test = edge_data.clone()[:, mask_test]

    return edge_train, edge_valid, edge_test, edge_data

nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

paper_c_paper_train, paper_c_paper_valid, paper_c_paper_test, paper_c_paper = process_edges(
    data, ('paper', 'cites', 'paper'), nums_train, nums_valid, nums_test,mask_position='both')

paper_t_field_train, paper_t_field_valid, paper_t_field_test, paper_t_field = process_edges(
    data, ('paper', 'has_topic', 'field_of_study'), nums_train, nums_valid, nums_test,mask_position='source')

author_w_paper_train, author_w_paper_valid, author_w_paper_test, author_w_paper = process_edges(
    data, ('author', 'writes', 'paper'), nums_train, nums_valid, nums_test,mask_position='target')

nums_test_author = torch.tensor(author_w_paper_test[0].unique())
nums_valid_author = torch.tensor(author_w_paper_valid[0].unique())
nums_train_author = torch.tensor(author_w_paper_train[0].unique())

author_a_institution_train, author_a_institution_valid, author_a_institution_test, author_a_institution = process_edges(
    data, ('author', 'affiliated_with', 'institution'), nums_train_author, nums_valid_author, nums_test_author,mask_position='source')

/tmp/ipykernel_61585/4219562739.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data, _ = torch.load(r"/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/datas